In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files

The data for this exercise is available at: https://www.kaggle.com/datamunge/sign-language-mnist/home

Sign up and download to find 2 CSV files: sign_mnist_test.csv and sign_mnist_train.csv -- You will upload both of them using this button before you can continue.


In [2]:
uploaded=files.upload()

Saving sign_mnist_test.csv to sign_mnist_test.csv
Saving sign_mnist_train.csv to sign_mnist_train.csv


In [5]:
def get_data(filename):
  # You will need to write code that will read the file passed
  # into this function. The first line contains the column headers
  # so you should ignore it
  # Each successive line contians 785 comma separated values between 0 and 255
  # The first value is the label
  # The rest are the pixel values for that picture
  # The function will return 2 np.array types. One with all the labels
  # One with all the images
  #
  # Tips: 
  # If you read a full line (as 'row') then row[0] has the label
  # and row[1:785] has the 784 pixel values
  # Take a look at np.array_split to turn the 784 pixels into 28x28
  # You are reading in strings, but need the values to be floats
  # Check out np.array().astype for a conversion
  with open(filename) as training_file:
    # Your code starts here
      
      labels = []
      images = []
      i = 0
      for indx, row in enumerate(training_file.readlines()):
          if indx == 0:
              continue
          row = row.split(',')
          labels.append(row[0])
          images_tmp = [float(value) for value in row[1:785]]
          images.append(np.array_split(images_tmp, 28))
      labels = np.array(labels).astype(float)
      images = np.array(images).astype(float)
    # Your code ends here
  return images, labels


training_images, training_labels = get_data('sign_mnist_train.csv')
testing_images, testing_labels = get_data('sign_mnist_test.csv')

# Keep these
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

# Their output should be:
# (27455, 28, 28)
# (27455,)
# (7172, 28, 28)
# (7172,)

(27455, 28, 28)
(27455,)
(7172, 28, 28)
(7172,)


In [6]:
# In this section you will have to add another dimension to the data
# So, for example, if your array is (10000, 28, 28)
# You will need to make it (10000, 28, 28, 1)
# Hint: np.expand_dims

training_images = np.expand_dims(training_images, axis=3)# Your Code Here
training_labels = tf.keras.utils.to_categorical(training_labels)
testing_images = np.expand_dims(testing_images, axis=3) # Your Code Here
testing_labels = tf.keras.utils.to_categorical(testing_labels)

# Create an ImageDataGenerator and do Image Augmentation
train_datagen = ImageDataGenerator(
    # Your Code Here
    rescale = 1./255.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )
traing_generator = train_datagen.flow(training_images, training_labels)

validation_datagen = ImageDataGenerator(
    rescale=1./255)
validation_generator = validation_datagen.flow(testing_images, testing_labels)
    # Your Code Here)
    
# Keep These
print(training_images.shape)
print(testing_images.shape)
    
# Their output should be:
# (27455, 28, 28, 1)
# (7172, 28, 28, 1)

(27455, 28, 28, 1)
(7172, 28, 28, 1)


In [10]:
next(traing_generator)

(array([[[[0.7568628 ],
          [0.7568628 ],
          [0.7568628 ],
          ...,
          [0.621822  ],
          [0.57931554],
          [0.5515711 ]],
 
         [[0.7568628 ],
          [0.7568628 ],
          [0.7568628 ],
          ...,
          [0.6145713 ],
          [0.5727239 ],
          [0.5482753 ]],
 
         [[0.7568628 ],
          [0.7568628 ],
          [0.7568628 ],
          ...,
          [0.6073205 ],
          [0.5661323 ],
          [0.5448846 ]],
 
         ...,
 
         [[0.16214049],
          [0.17579   ],
          [0.11935601],
          ...,
          [0.4431501 ],
          [0.43279344],
          [0.44137028]],
 
         [[0.17302082],
          [0.16253199],
          [0.04853032],
          ...,
          [0.42009372],
          [0.4060839 ],
          [0.39979643]],
 
         [[0.1680998 ],
          [0.10118596],
          [0.03822663],
          ...,
          [0.42999643],
          [0.4275303 ],
          [0.42613807]]],
 
 
        [

In [ ]:
# Define the model
# Use no more than 2 Conv2D and 2 MaxPooling2D
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(25, activation='softmax')]
    )

# Compile Model. 
model.compile(loss='categorical_crossentropy', optimize='rmsprop', metrics=['acc'])# Your Code Here)

# Train the Model
history = model.fit_generator(traing_generator, 
                              epochs=10, 
                              steps_per_epoch=100,
                              validation_data = validation_generator,
                              verbose=1)
                              # Your Code Here (set 'epochs' = 2))

model.evaluate(testing_images / 255., testing_labels, verbose=0)
    
# The output from model.evaluate should be close to:
[6.92426086682151, 0.56609035]


In [ ]:
# Plot the chart for accuracy and loss on both training and validation

import matplotlib.pyplot as plt
acc = history.history['acc']# Your Code Here
val_acc = history.history['val_acc']# Your Code Here
loss = history.history['loss']# Your Code Here
val_loss = history.history['val_loss']# Your Code Here

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()